In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lrex').getOrCreate()

22/05/24 15:24:12 WARN Utils: Your hostname, ls-Aspire-A515-51G resolves to a loopback address: 127.0.1.1; using 192.168.15.8 instead (on interface wlp3s0)
22/05/24 15:24:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/24 15:24:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/24 15:24:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/05/24 15:25:21 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [4]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
lr = LinearRegression(featuresCol='features', labelCol='label', 
                     predictionCol='prediction')

In [6]:
lr_model = lr.fit(training)

22/05/24 15:27:58 WARN Instrumentation: [ce72d60d] regParam is zero, which might cause numerical instability and overfitting.


In [7]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lr_model.intercept

0.14228558260358093

In [9]:
training_summary = lr_model.summary

In [26]:
training_summary

In [10]:
training_summary.r2 # How much variance is explained by our model

0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

In [12]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

22/05/24 15:30:58 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [13]:
split_object = all_data.randomSplit([0.7, 0.3]) # train: 70 %, test: 30 %

In [14]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [15]:
train_data, test_data = split_object

In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
+----------

In [17]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                366|
|   mean|-0.2704128425469907|
| stddev| 10.203379022648761|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [18]:
correct_model = lr.fit(train_data)

22/05/24 15:33:32 WARN Instrumentation: [9b19d920] regParam is zero, which might cause numerical instability and overfitting.


In [19]:
test_results = correct_model.evaluate(test_data)

In [20]:
test_results.residuals.show() # y_real - y_pred

/opt/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
|-26.570475070625953|
|-26.480803405617767|
|-22.418953797280498|
| -18.29808580622997|
| -20.39259118048942|
| -15.57831601452615|
|-18.609925439163334|
|-14.859915370919513|
|-13.431507334813908|
| -9.587849257213886|
|-13.658208550338637|
|  -13.1079426574707|
|-14.688009021993885|
|-12.840034754975536|
| -14.25858442787605|
| -5.003931328713672|
| -8.070843269772057|
| -9.038162419472151|
|-10.539074280854559|
|-10.264717050200156|
+-------------------+
only showing top 20 rows



In [21]:
unlabeled_data = test_data.select('features')

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [23]:
predictions = correct_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.4755429671506815|
|(10,[0,1,2,3,4,5,...|-0.3246800228653047|
|(10,[0,1,2,3,4,5,...|-1.0684863236560145|
|(10,[0,1,2,3,4,5,...|  -4.53937461068937|
|(10,[0,1,2,3,4,5,...| 0.3351085647002076|
|(10,[0,1,2,3,4,5,...|-1.7484047181497988|
|(10,[0,1,2,3,4,5,...| 2.3484951637106004|
|(10,[0,1,2,3,4,5,...|  -1.09159719487506|
|(10,[0,1,2,3,4,5,...| 0.8729315459577192|
|(10,[0,1,2,3,4,5,...|-2.8798071238189737|
|(10,[0,1,2,3,4,5,...| 1.4601119856772249|
|(10,[0,1,2,3,4,5,...| 1.2297751575038594|
|(10,[0,1,2,3,4,5,...| 3.2562066564534256|
|(10,[0,1,2,3,4,5,...|  2.256723455107405|
|(10,[0,1,2,3,4,5,...|   3.90266582750137|
|(10,[0,1,2,3,4,5,...| -5.037421783866784|
|(10,[0,1,2,3,4,5,...|-1.7375434327926007|
|(10,[0,1,2,3,4,5,...| -0.713460188312931|
|(10,[0,1,2,3,4,5,...| 1.0490644020300113|
|(10,[0,1,2,3,4,5,...| 1.0536504791179089|
+----------